In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader
from torchsummary import summary
from tqdm import tqdm


# MNIST Classification problem

This is a notebook to perform the MNIST Classification Problem by using a Convolutional Neural Network. The scope of the problem is to classify properly handwritten numbers. We have 60000 numbers on our Training set, 10000 numbers on our test set. 

We also have 10 classes, whose are the numbers from 0 to 9. 

## PyTorch Implementation:

For achieving this task, we will use this notebook to learn how to use the different implementations of PyTorch. We will need installed PyTorch, some parts of the PyTorch Ecosystem, and torchsummary. Also, we will need to install Scikit-Learn for showing the Confusion Matrix. 

Our first task is to load the Data. 

<b>REMEMBER ALWAYS TO HAVE YOUR GPU WORKING</b>

### Links to the Documentation:

- <a href="https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST">Here</a> the user will be able to find how to use the MNIST object from PyTorch. The user could also download the data directly from <a href="http://yann.lecun.com/exdb/mnist/index.html">Yann LeCun's repository.</a>

- For accessing the documentation for the random splitter, the user can refer to <a href="https://pytorch.org/docs/stable/data.html#"> documentation.</a> Note that the documentation is for the data library from PyTorch, which has inside the random splitter. 

- For accessing to the DataLoader documentation, you can follow   <a href="https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader"> this link.</a>

### Loading the Data:

In [ ]:
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
])

mnist_dataset = torchvision.datasets.MNIST(
    root=image_path,
    train=True,
    transform=transform,
    download=True
)
mnist_test_dataset = datasets.MNIST(
    root=image_path,
    train=False,
    transform=transform,
    download=False
)
# Train/ Validation Split (I am using the splitter from PyTorch.)
mnist_train_dataset, mnist_val_dataset = random_split(mnist_dataset, [50000, 10000])

# Call DataLoaders:
train_dl = DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)

val_dl = DataLoader(mnist_val_dataset, batch_size=64, shuffle=True)

# As I'm not using the test set sequentially, I won't put test set in a DataLoader.

## Creating a Model with PyTorch:

For this problem, I am using the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html"> nn.Sequential() class</a> to create a model. Also, I am inheriting all the methods from the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html"> nn.Module() object</a>. 

Note that I will also train my model by parallelizing computations with a GPU. This is done by calling model().to(device).

Finally, note that as an output for the following cell, we can see a summary of the model, similar to the summary provided by Keras.

In [ ]:
# Model creation WITHOUT DROPOUT LAYER!
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels= 64, out_channels=128, kernel_size=3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 10)
        )
    
    def forward(self, input):
        return self.main(input)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model().to(device)

# Summary:
summary(model.main, input_size=(1,28, 28), batch_size=64)
# Selection of Loss and Optimizer:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 32, 26, 26]             320
              ReLU-2           [64, 32, 26, 26]               0
         MaxPool2d-3           [64, 32, 13, 13]               0
            Conv2d-4           [64, 64, 11, 11]          18,496
              ReLU-5           [64, 64, 11, 11]               0
         MaxPool2d-6             [64, 64, 5, 5]               0
            Conv2d-7            [64, 128, 3, 3]          73,856
              ReLU-8            [64, 128, 3, 3]               0
         MaxPool2d-9            [64, 128, 1, 1]               0
          Flatten-10                  [64, 128]               0
           Linear-11                  [64, 128]          16,512
             ReLU-12                  [64, 128]               0
          Dropout-13                  [64, 128]               0
           Linear-14                  [

## Training of the Model

More than trying to achieve the highest possible score, on this notebook we are aiming to regularize the model by adding dropout layers and performing EarlyStopping during the training of the network. 

Each part of the training function is commented to aid the easy reading of the code.

In [ ]:
# Training Script for Classification tasks, with EarlyStopping:


def train(model, train_dl, num_epochs=100, patience=5):

    # Lists for storing the training values:
    training_loss = []
    training_acc = []
    val_loss = []
    val_acc = []
    # Control for the Early Stopping:
    best_acc = 0
    counter = 0

    for epoch in range(num_epochs):
        # Initialize the running loss and accuracy for estimating the metrics 
        # during the running of each batch. 
        running_loss = 0
        running_acc = 0

        for x_batch, y_batch in tqdm(train_dl):
            # Prediciton:
            pred = model(x_batch.to(device))
            # Computing the Loss:
            loss = loss_fn(pred, y_batch.to(device))
            # We want to get an estimate of the loss per epoch
            running_loss += loss.item()/len(train_dl)
            # Backpropagation of the loss:
            loss.backward()
            # Actualization of the weights:
            optimizer.step()
            # Setting to zero the grads.
            optimizer.zero_grad()
            # Computing running accuracy.
            correct = (torch.argmax(pred, dim=1) == y_batch.to(device)).float()
            running_acc += correct.sum()/len(train_dl.dataset)
        # History of the loss and accuracy per Epoch:
        training_loss.append(running_loss)
        training_acc.append(running_acc.cpu())
        print(
            f'Epoch {epoch+1} / {num_epochs}. Training Loss = {running_loss: .4f} '
            f'Training_acc = {running_acc : .4f}')

        # Doing the same for the validation.
        # (Difference: We don't compute the gradients)

        running_loss = 0
        running_acc = 0
        for x_batch, y_batch in val_dl:
            pred = model(x_batch.to(device))
            loss = loss_fn(pred, y_batch.to(device))
            running_loss += loss.item()/len(train_dl)
            correct = (torch.argmax(pred, dim=1) == y_batch.to(device)).float()
            running_acc += correct.sum()/len(val_dl.dataset)
        val_loss.append(running_loss)
        val_acc.append(running_acc.cpu())
        print(
            f'Val_Loss = {running_loss: .4f} '
            f'Val_acc = {running_acc : .4f}')
        
        # Early Stopping Criteria, note that if the accuracy is not improving
        # after several epochs (given by the patience), it will stop and save
        # the model that is giving best results on unseen data. 
        
        if float(running_acc) < float(best_acc):
            counter +=1
            if counter > patience:
                print('Early stopping achieved.')
                model=snapshot
                return training_loss, training_acc, val_loss, val_acc
        else:
            counter = 0
            best_acc = running_acc
            snapshot = model
            
        print(f'Best val_acc = {best_acc}')        
        print(f'counter is {counter}')
        
    return training_loss, training_acc, val_loss, val_acc


hist = train(model, num_epochs=100, patience=10, train_dl=train_dl)

100%|██████████| 782/782 [00:15<00:00, 51.26it/s]


Epoch 1 / 100. Training Loss =  0.6271 Training_acc =  0.7911
Val_Loss =  0.0401 Val_acc =  0.9485
Best val_acc = 0.9484997987747192
counter is 0


100%|██████████| 782/782 [00:07<00:00, 97.85it/s] 


Epoch 2 / 100. Training Loss =  0.1562 Training_acc =  0.9599
Val_Loss =  0.0237 Val_acc =  0.9694
Best val_acc = 0.9693995714187622
counter is 0


100%|██████████| 782/782 [00:07<00:00, 99.97it/s] 


...

Epoch 48 / 100. Training Loss =  0.0101 Training_acc =  0.9978
Val_Loss =  0.0259 Val_acc =  0.9871
Best val_acc = 0.9890987873077393
counter is 9


100%|██████████| 782/782 [00:07<00:00, 101.43it/s]


Epoch 49 / 100. Training Loss =  0.0105 Training_acc =  0.9978
Val_Loss =  0.0245 Val_acc =  0.9854
Best val_acc = 0.9890987873077393
counter is 10


100%|██████████| 782/782 [00:08<00:00, 96.79it/s]


Epoch 50 / 100. Training Loss =  0.0106 Training_acc =  0.9977
Val_Loss =  0.0213 Val_acc =  0.9883
Early stopping achieved.


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(hist[0], 'r--',lw=2, label='Training Loss')
plt.plot(hist[2], 'k',lw=2, label='Validation Loss')
plt.legend(loc=1)
plt.title('Loss')
plt.xlabel('Epochs')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(hist[1], 'r--',lw=2, label='Training Accuracy')
plt.plot(hist[3], 'k',lw=2, label='Validation Accuracy')
plt.legend(loc=4)
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.show()

In [ ]:
model.to('cpu')
pred = torch.argmax(model((mnist_test_dataset.data.unsqueeze(1)/255)), dim=1)

correct = (pred == mnist_test_dataset.targets).float()
test_acc = correct.sum()/len(mnist_test_dataset.targets)
conf_matrix_1 = confusion_matrix(mnist_test_dataset.targets, pred)
fig, ax = plt.subplots(figsize=(9, 9))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_1,)
disp.plot(ax=ax, cmap=plt.cm.magma, colorbar=False)
plt.title(f'The Test Accuracy is : {test_acc*100 : .4f} %')
plt.savefig('conf_MNIST_cnn_no_dropout.jpg', bbox_inches='tight')
plt.show()

In [ ]:
model

model(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=128, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.3, inplace=False)
    (13): Linear(in_features=128, out_features=128, bias=True)
    (14): ReLU()
    (15): Dropout(p=0.3, inplace=False)
    (16): Linear(in_features=128, out_features=128, bias=True)
    (17): ReLU()
    (18): Dropout(p=0.3, inplace=False)
    (19): Linear(in_features=128, out_features=